<a href="https://colab.research.google.com/github/evilsizord/mscs-data-mining-hw2/blob/main/Data_Mining_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Naive Bayes Classifier
Daniel Evilsizor \
November 13, 2022

In [2]:
# Get Data from Kaggle.com

# NOTE: Requires you to have a Kaggle.com account. From your account you can generate an API key.
# It will be provided in kaggle.json. Upload the JSON file to this project BEFORE RUNNING.

# src: https://www.kaggle.com/general/74235

! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download gaveshjain/ford-sentence-classifiaction-dataset   # yes there is a typo in the actual URL
! mkdir sentence-dataset
! unzip ford-sentence-classifiaction-dataset.zip -d sentence-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/2.92M [00:00<?, ?B/s]
100% 2.92M/2.92M [00:00<00:00, 161MB/s]
mkdir: cannot create directory ‘sentence-dataset’: File exists
Archive:  ford-sentence-classifiaction-dataset.zip
  inflating: sentence-dataset/sample_submission.csv  
  inflating: sentence-dataset/test_data.csv  
  inflating: sentence-dataset/train_data.csv  


In [26]:
# load all necessary libraries

import os
import pandas
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [45]:
#from numpy import nan
import numpy as np

# Load and preview the data
input_train = pandas.read_csv('sentence-dataset/train_data.csv')
input_test = pandas.read_csv('sentence-dataset/test_data.csv')

# throw out the integer indexs before merging
input_train = input_train.drop(input_train.columns[[0]], axis=1)
input_test = input_test.drop(input_test.columns[[0]], axis=1)

all_data = pandas.concat([input_train, input_test])
# how many rows?
print(len(all_data), 'total rows loaded')

# throw out null sentences (see https://stackoverflow.com/a/56708633)
# todo: sigh why this not working..
#all_data = all_data[ all_data['New_Sentence'].str.strip().astype(bool) & all_data['New_Sentence'] != nan ]
#print(len(all_data), 'rows after removing empty')

all_data['New_Sentence'].replace('', np.nan, inplace=True)
all_data.dropna(subset=['New_Sentence'], inplace=True)
print(len(all_data), 'rows after removing empty 2')

#debug: what are values for type?
#print(all_data['Type'].unique())

testdata = all_data[ all_data['Type'].isnull() ]
traindevdata = all_data[ all_data['Type'].notnull() ]

split = int(.5*len(traindevdata))
traindata = traindevdata[:split]
devdata = traindevdata[(split+1):]

print(len(traindata), 'training,', len(devdata), 'dev,', len(testdata), 'test rows loaded')

# preview the result
all_data.head(5)

75144 total rows loaded
75144 rows after removing empty
73750 rows after removing empty
['Responsibility' 'Requirement' 'Skill' 'SoftSkill' 'Education'
 'Experience' nan]
29501 training, 29500 dev, 14748 test rows loaded


,Sentence_id,New_Sentence,Type
0,GERRES15609,Author and/or Review architecture/design and o...,Responsibility
1,PHERES15784,Should be able to develop custom dynamic shape...,Responsibility
2,GERREQ10457,Experience in working crosslly with a larger ...,Requirement
3,GERSKL27235,"Previous business experience, including but no...",Skill
4,HONSSK18415,Delivering fast and right the first time.,SoftSkill


In [46]:
# get vocabulary statistics
vocab = {}
ps = PorterStemmer()
for index, row in all_data.iterrows():
  if index == 5:
    #print(row)
    print(row['New_Sentence'])
    print ('New_Sentence' in row)
  #tokens = []
  
  if 'New_Sentence' in row:
    tokens = word_tokenize(row['New_Sentence'])

    # todo: remove punctuation
    # todo: remove stopwords?
    # todo: do we need to lowercase the tokens?
    for tok in tokens:
      stemword = ps.stem(tok)
      if stemword in vocab:
        vocab[stemword] += 1
      else:
        vocab[stemword] = 1

# preview
print('Found', len(vocab.keys()), 'words in vocabulary')

Track  department expenses and capital budget .
True
Assure compliance and recommend actions to improve the effectiveness of  the quality system.
True
Found 24757 words in vocabulary


## References

CSV reader example code:\
https://realpython.com/python-csv/

https://www.geeksforgeeks.org/removing-stop-words-nltk-python/

https://www.geeksforgeeks.org/python-stemming-words-with-nltk/?ref=lbp

https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
